In [1]:
import tensorflow as tf
import numpy as np
import re
import string

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  20.1M      0  0:00:03  0:00:03 --:--:-- 20.1M


In [3]:
!ls aclImdb

imdbEr.txt  imdb.vocab	README	test  train


In [4]:
!ls aclImdb/test

labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt


In [5]:
!ls aclImdb/train

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [6]:
!cat aclImdb/train/pos/6248_7.txt

Being an Austrian myself this has been a straight knock in my face. Fortunately I don't live nowhere near the place where this movie takes place but unfortunately it portrays everything that the rest of Austria hates about Viennese people (or people close to that region). And it is very easy to read that this is exactly the directors intention: to let your head sink into your hands and say "Oh my god, how can THAT be possible!". No, not with me, the (in my opinion) totally exaggerated uncensored swinger club scene is not necessary, I watch porn, sure, but in this context I was rather disgusted than put in the right context.<br /><br />This movie tells a story about how misled people who suffer from lack of education or bad company try to survive and live in a world of redundancy and boring horizons. A girl who is treated like a whore by her super-jealous boyfriend (and still keeps coming back), a female teacher who discovers her masochism by putting the life of her super-cruel "lover" 

In [7]:
!rm -r aclImdb/train/unsup

In [8]:
!ls aclImdb/train

labeledBow.feat  pos		urls_neg.txt  urls_unsup.txt
neg		 unsupBow.feat	urls_pos.txt


In [9]:
batch_size = 32

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
                  "aclImdb/train",
                  batch_size = batch_size,
                  validation_split = 0.2,
                  subset = "training",
                  seed = 1337
)

raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
                  "aclImdb/train",
                  batch_size = batch_size,
                  validation_split = 0.2,
                  subset = "validation",
                  seed = 1337
)

raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
                  "aclImdb/test",
                  batch_size = batch_size
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [10]:
tf.data.experimental.cardinality(raw_train_ds)

<tf.Tensor: shape=(), dtype=int64, numpy=625>

In [11]:
for text_batch, text_label in raw_train_ds.take(1):
  for i in range(5):
    print (text_batch.numpy()[i])
    print (text_label.numpy()[i])

b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

In [12]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase,"<br />"," ")
  return tf.strings.regex_replace(stripped_html,"[%s]" %re.escape(string.punctuation),"")

max_features = 20000
embedding_dim = 128
sequence_length = 500

vectorize_layer = TextVectorization(
                  standardize = custom_standardization,
                  max_tokens=max_features,
                  output_mode = 'int',
                  output_sequence_length = sequence_length
)

text_ds = raw_train_ds.map(lambda x,y:x)
vectorize_layer.adapt(text_ds)

In [13]:
def vectorize_text(text,label):
  text = tf.expand_dims(text,-1)
  return vectorize_layer(text),label

train_ds = raw_train_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)

train_ds = train_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)

In [14]:
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(None,),dtype='int64')

x = layers.Embedding(max_features,embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

x = layers.Conv1D(128,7,padding='valid',activation='relu',strides=3)(x)
x = layers.Conv1D(128,7,padding='valid',activation='relu',strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

x = layers.Dense(128,activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(1,activation="sigmoid",name="predictions")(x)

model = tf.keras.Model(inputs,predictions)

model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [17]:
epoch = 5

model.fit(train_ds,validation_data=val_ds,epochs=epoch)

Epoch 1/5
625/625 [==============================] - 125s 198ms/step - loss: 0.6131 - accuracy: 0.5962 - val_loss: 0.3560 - val_accuracy: 0.8506
Epoch 2/5
625/625 [==============================] - 119s 191ms/step - loss: 0.2710 - accuracy: 0.8919 - val_loss: 0.3218 - val_accuracy: 0.8766
Epoch 3/5
625/625 [==============================] - 119s 190ms/step - loss: 0.1353 - accuracy: 0.9501 - val_loss: 0.3495 - val_accuracy: 0.8744
Epoch 4/5
625/625 [==============================] - 119s 190ms/step - loss: 0.0838 - accuracy: 0.9723 - val_loss: 0.8436 - val_accuracy: 0.8060
Epoch 5/5
625/625 [==============================] - 118s 189ms/step - loss: 0.0619 - accuracy: 0.9783 - val_loss: 0.6233 - val_accuracy: 0.8518


In [18]:
model.evaluate(test_ds)

782/782 [==============================] - 34s 43ms/step - loss: 0.6307 - accuracy: 0.8418


[0.6307169795036316, 0.8417999744415283]

**End to End Model**

In [19]:
inputs = tf.keras.Input(shape=(1,),dtype="string")
indices = vectorize_layer(inputs)
outputs = model(indices)

end_to_end_model = tf.keras.Model(inputs,outputs)
end_to_end_model.complie(loss="binary_cross_entropy",optimizer="adam",metrics=["accuracy"])

end_to_end_model.evaluate(raw_test_ds)

AttributeError: ignored